# Blending optimization: Peanut butter production

## Back story
A peanut butter house produces its product by blending different types of peanuts to match the target protein and fat contents.

A quality peanut butter should contain at least 20% protein and between 48--52% fat in a kilogram.

## Data

| Peanut type | Cost ($/kg) | Protein content (%) | Fat content (%) |
|:---:|:---:|:---:|:---:|
| Type A | 4.00 | 26 | 50 |
| Type B | 3.00 | 20 | 55 |
| Type C | 2.50 | 18 | 40 |

## Modeling

### Indices and parameters
- $i \in \{A,B,C\}$
- $C_{i}$, the cost of peanut of type $i$.
- $P_{i}$, the protein content of peanut of type $i$.
- $P^{-}$, the lower target protein content.
- $F_{i}$, the fat content of peanut of type $i$.
- $F^{-}$, the lower target fat content.
- $F^{+}$, the upper target fat content.

### Decision variables
- $x_{i} \geq 0$, proportion (kg) of peanut of type $i$ in the mix.

### Objective function
- To minimize the total cost, which is defined by
$$ 
Cost = 4x_{A} + 3x_{B} + 2.5x_{C}
$$

### Constraints
- $x_{A} + x_{B} + x_{C} = 1$, the mix comprises to a full proportion of a kilogram.
- $26x_{A} + 20X_{B} + 18x_{C} \geq P^{-}$, the protein content is at least $22\%$.
- $F^{-} \leq 50x_{A} + 55x_{B} + 50x_{C} \leq F^{+}$, the fat content is between $48\%$ and $52\%$.

In [ ]:
import numpy as np
from pyscipopt import Model, quicksum

In [ ]:
types = ['A', 'B', 'C']
n = len(types)
N = range(n)
C = np.array([4, 3, 2.5])
P = np.array([26, 20, 18])
F = np.array([50, 55, 40])
P_lower = 22
F_lower = 48
F_upper = 52

m = Model()
x = [m.addVar(name=f'x_{i}', vtype='C', lb=0) for i in N]
m.setObjective( quicksum(C[i]*x[i] for i in N), sense='minimize')
m.addCons( quicksum(x[i] for i in N) == 1 )
m.addCons( quicksum(P[i]*x[i] for i in N) >= P_lower )
m.addCons( quicksum(F[i]*x[i] for i in N) >= F_lower )
m.addCons( quicksum(F[i]*x[i] for i in N) <= F_upper );

In [ ]:
m.optimize()

In [ ]:
SOL = m.getBestSol()
OBJ = m.getObjVal()
x_opt = np.array([SOL[x[i]] for i in N])

print(f"The optimal formula for 1kg of peanut butter includes")
for i in N: print(f"- {x_opt[i]*100}% of Type {types[i]} peanuts")
print(f"The formula contains {np.sum(P*x_opt)}% of protein and {np.sum(F*x_opt)}% of fat.")
print(f"The cost for producing 1kg of this peanut butter is {OBJ}$.")

---